In [ ]:
import pandas as pd
import gym
import numpy as np
import tensorflow as tf
from collections import deque
import random as rnd
from tensorflow import keras
from tensorflow.keras import layers, models

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


Пока мы ещё маленькие и совсем ничего не понимаем в нейросетях. Сделаем простую игру (что-то в духе камень-ножницы-бумага), чтобы проверить, работает ли модель в принципе и научится ей пользоваться для более сложной задачи, которая ждёт нас впереди.

In [ ]:
X = np.array([
    [a, b] for _ in range(1500)
    for a, b in [(rnd.randint(1, 3), rnd.randint(1, 3))]
    if a != b
]) # матчи

y = np.array([0 if a < b else 1 for a, b in X]) # номер победившего в паре игрока

model = models.Sequential([
    layers.InputLayer(input_shape=(2,)),  # вход: пара героев
    layers.Dense(16, activation='relu'),  # скрытый слой
    layers.Dense(1, activation='sigmoid')  # выход: победитель (0 или 1)
])

# компилируем модель
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# обучаем модель
model.fit(X, y, epochs=50, batch_size=10)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7066 - loss: 0.5500
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 0.3964
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - loss: 0.2801
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 0.1880
Epoch 5/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.1231
Epoch 6/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0854
Epoch 7/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0624
Epoch 8/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0444
Epoch 9/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0340
Epoch 10/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0268
Epoch 11/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0214
Epoch 12/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

In [ ]:
X_test = np.array([
    [a, b] for _ in range(10)
    for a, b in [(rnd.randint(1, 3), rnd.randint(1, 3))]
    if a != b
])

Y_test = np.array([0 if a < b else 1 for a, b in X_test])  # правильные ответы

y_pred = model.predict(X_test) # предсказания модели
y_pred = (y_pred > 0.5).astype(int)  # преобразуем в 0 или 1

errors = 0

for i, prediction in enumerate(y_pred):
    p1 = ['A', 'B', 'C'][X_test[i][0] - 1]
    p2 = ['A', 'B', 'C'][X_test[i][1] - 1]
    if prediction != Y_test[i]:
        print(f"Ошибка в игре {i+1}: {p1} против {p2}, победил не {p1 if prediction == 0 else p2}")
        errors += 1
    else:
        print(f"Игра {i+1}: {p1} против {p2} => Победитель: {p1 if prediction == 0 else p2}")

print(f"Процент ошибок: {((errors / len(Y_test)) * 100):.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Игра 1: B против C => Победитель: B
Игра 2: C против A => Победитель: A
Игра 3: B против A => Победитель: A
Игра 4: B против C => Победитель: B
Игра 5: A против B => Победитель: A
Игра 6: A против C => Победитель: A
Игра 7: B против A => Победитель: A
Процент ошибок: 0.00%


Всё работает, отлично - модель научилась угадывать победителя. Теперь напишем, чтобы она сама участвовала в игре и выбирала себе героя.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

def reward(a, b):
    return ([a, b] in [[0, 1], [0, 2], [1, 2]])

# softmax модель
model = keras.Sequential([
    keras.layers.Input(shape=(1,)),
    keras.layers.Dense(3, activation='softmax')
])

optimizer = keras.optimizers.Adam(learning_rate=0.01)

num_episodes = 300   # эпохи обучения
steps_per_ep = 50    # матчей за одну эпоху

for episode in range(num_episodes):
    with tf.GradientTape() as tape:
        state_input = tf.ones((steps_per_ep, 1), dtype=tf.float32)
        probs = model(state_input, training=True)  # вероятности
.
        log_probs = tf.math.log(probs + 1e-6)

        sampled_actions = tf.random.categorical(log_probs, num_samples=1) # ходы модели
        sampled_actions = tf.squeeze(sampled_actions, axis=1)

        rewards = []
        for i in range(steps_per_ep):
            model_hero = sampled_actions[i].numpy()
            possible_opp = [x for x in [0,1,2] if x != model_hero]
            opp_hero = np.random.choice(possible_opp)
            rewards.append(reward(model_hero, opp_hero))

        rewards = tf.constant(rewards, dtype=tf.float32)

        chosen_action_log_p = tf.reduce_sum(
            tf.one_hot(sampled_actions, depth=3) * log_probs,
            axis=1
        ) # корректируем модель
        loss = -tf.reduce_mean(rewards * chosen_action_log_p)

    # обновляем веса
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # статистика
    if (episode + 1) % 20 == 0:
        mean_reward = tf.reduce_mean(rewards)
        print(f"Episode {episode+1}/{num_episodes} | Mean reward: {mean_reward.numpy():.2f} | Loss: {loss.numpy():.3f}")

Episode 20/300 | Mean reward: 0.46 | Loss: 0.509
Episode 40/300 | Mean reward: 0.40 | Loss: 0.430
Episode 60/300 | Mean reward: 0.56 | Loss: 0.522
Episode 80/300 | Mean reward: 0.68 | Loss: 0.526
Episode 100/300 | Mean reward: 0.86 | Loss: 0.440
Episode 120/300 | Mean reward: 0.88 | Loss: 0.238
Episode 140/300 | Mean reward: 0.96 | Loss: 0.259
Episode 160/300 | Mean reward: 0.88 | Loss: 0.127
Episode 180/300 | Mean reward: 0.96 | Loss: 0.120
Episode 200/300 | Mean reward: 0.98 | Loss: 0.114
Episode 220/300 | Mean reward: 0.98 | Loss: 0.186
Episode 240/300 | Mean reward: 1.00 | Loss: 0.188
Episode 260/300 | Mean reward: 0.98 | Loss: 0.026
Episode 280/300 | Mean reward: 0.98 | Loss: 0.022
Episode 300/300 | Mean reward: 1.00 | Loss: 0.020


Практика: смотрим, сколько матчей выиграет наша обученная модель

In [ ]:
N_test = 1000
test_input = tf.ones((N_test, 1), dtype=tf.float32)
test_probs = model(test_input, training=False)   # (N_test, 3)
test_log_probs = tf.math.log(test_probs + 1e-10)
test_actions = tf.random.categorical(test_log_probs, num_samples=1)
test_actions = tf.squeeze(test_actions, axis=1).numpy()

wins = 0
for i in range(N_test):
    L = ['A','B','C']
    model_hero = test_actions[i]
    opp_hero = np.random.choice([x for x in [0,1,2] if x != model_hero])
    if reward(model_hero, opp_hero):
        wins += 1
        if i % 50 == 0:
            print(f'Модель с героем {L[model_hero]} обыграла героя соперника {L[opp_hero]}')
    else:
        if i % 50 == 0:
            print(f'x - Модель с героем {L[model_hero]} проиграла герою соперника {L[opp_hero]}')

win_percent = 100.0 * wins / N_test
print(f"\nИтоговая проверка: модель выиграла в {win_percent:.2f}% случаев.")

Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника B
Модель с героем A обыграла героя соперника B
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника C
x - Модель с героем C проиграла герою соперника A
Модель с героем B обыграла героя соперника C
Модель с героем A обыграла героя соперника B
Модель с героем A обыграла героя соперника B
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника C
Модель с героем A обыграла героя соперника B

Итоговая проверка: модель выиграла в 98.10% случаев.


УРААААА ГОООООЛ, он что-то выиграл.
Это здорово, что он придумал невероятную тактику всегда выбирать героя А, но что, если правила С будет бить А, а не наоборот. Да и вообще, пора матрицу игр писать, ничью добавить...
А ещё устроим дополнительную проверку модели на адекватность - нужно будет выбирать свой ход исходя из хода соперника ("если у соперника ножницы, то очевидно нужен камень", и т.д.)

p.s. можно заметить, что это уже немного похоже на задачу, которую нам предстоит решить

ОБУЧЕНИЕ

In [ ]:
matrix = [[0.5, 1, 0], [0, 0.5, 1], [1, 0, 0.5]]
def reward(a, b):
    return matrix[a][b]

model = keras.Sequential([
    keras.layers.Input(shape=(1,)),
    keras.layers.Embedding(input_dim=3, output_dim=4),
    keras.layers.Flatten(),
    keras.layers.Dense(3, activation='softmax')
])

optimizer = keras.optimizers.Adam(learning_rate=0.01)

num_episodes = 300   # эпохи обучения
steps_per_ep = 50    # матчей за одну эпоху

for episode in range(num_episodes):
    with tf.GradientTape() as tape:
        opp_hero = np.random.randint(0, 3, size=steps_per_ep)

        state_input = tf.constant(opp_hero, dtype=tf.int32)
        state_input = tf.reshape(state_input, (steps_per_ep, 1))
        probs = model(state_input, training=True)
        log_probs = tf.math.log(probs + 1e-6)

        sampled_actions = tf.random.categorical(log_probs, num_samples=1)
        sampled_actions = tf.squeeze(sampled_actions, axis=1)

        rewards = []
        for i in range(steps_per_ep):
            model_hero = sampled_actions[i].numpy()
            rewards.append(reward(model_hero, opp_hero[i]))

        rewards = tf.constant(rewards, dtype=tf.float32)
        chosen_action_log_p = tf.reduce_sum(
            tf.one_hot(sampled_actions, depth=3) * log_probs,
            axis=1
        )
        loss = -tf.reduce_mean(rewards * chosen_action_log_p)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if (episode + 1) % 20 == 0:
        mean_reward = tf.reduce_mean(rewards)
        print(f"Episode {episode+1}/{num_episodes} | Mean reward: {mean_reward.numpy():.2f} | Loss: {loss.numpy():.3f}")

Episode 20/300 | Mean reward: 0.52 | Loss: 0.437
Episode 40/300 | Mean reward: 0.90 | Loss: 0.326
Episode 60/300 | Mean reward: 0.96 | Loss: 0.033
Episode 80/300 | Mean reward: 1.00 | Loss: 0.012
Episode 100/300 | Mean reward: 0.99 | Loss: 0.056
Episode 120/300 | Mean reward: 1.00 | Loss: 0.005
Episode 140/300 | Mean reward: 1.00 | Loss: 0.004
Episode 160/300 | Mean reward: 1.00 | Loss: 0.003
Episode 180/300 | Mean reward: 1.00 | Loss: 0.002
Episode 200/300 | Mean reward: 1.00 | Loss: 0.002
Episode 220/300 | Mean reward: 1.00 | Loss: 0.001
Episode 240/300 | Mean reward: 1.00 | Loss: 0.001
Episode 260/300 | Mean reward: 1.00 | Loss: 0.001
Episode 280/300 | Mean reward: 1.00 | Loss: 0.001
Episode 300/300 | Mean reward: 1.00 | Loss: 0.001


ПРОВЕРКА

In [ ]:
N_test = 1000
opp_hero_test = np.random.randint(0,3, size=N_test)
test_input = tf.constant(opp_hero_test, dtype=tf.int32)
test_input = tf.reshape(test_input, (N_test, 1))

test_probs = model(test_input, training=False)  # (N_test,3)
test_log_probs = tf.math.log(test_probs + 1e-6)
test_actions = tf.random.categorical(test_log_probs, num_samples=1)
test_actions = tf.squeeze(test_actions, axis=1).numpy()

wins = 0
L = ['A','B','C']
for i in range(N_test):
    model_hero = test_actions[i]
    opp_hero = opp_hero_test[i]
    r = reward(model_hero, opp_hero)
    wins += r
    if i % 50 == 0 or r < 1:
        if r == 1.0:
            print(f'Модель с героем {L[model_hero]} обыграла героя соперника {L[opp_hero]}')
        elif r:
            print(f'Модель с героем {L[model_hero]} сыграла вничью с героем соперника {L[opp_hero]}')
        else:
            print(f'x - Модель с героем {L[model_hero]} проиграла герою соперника {L[opp_hero]}')

win_percent = 100.0 * wins / N_test
print(f"\nИтоговая проверка: модель выиграла в {win_percent:.2f}% матчей.")

Модель с героем A обыграла героя соперника B
Модель с героем C обыграла героя соперника A
Модель с героем A обыграла героя соперника B
Модель с героем A обыграла героя соперника B
Модель с героем B обыграла героя соперника C
Модель с героем A обыграла героя соперника B
Модель с героем C обыграла героя соперника A
Модель с героем C обыграла героя соперника A
Модель с героем A обыграла героя соперника B
Модель с героем A обыграла героя соперника B
Модель с героем B обыграла героя соперника C
Модель с героем A обыграла героя соперника B
Модель с героем A обыграла героя соперника B
Модель с героем C обыграла героя соперника A
Модель с героем A сыграла вничью с героем соперника A
Модель с героем A обыграла героя соперника B
Модель с героем B обыграла героя соперника C
x - Модель с героем A проиграла герою соперника C
Модель с героем B обыграла героя соперника C
Модель с героем C обыграла героя соперника A
Модель с героем A обыграла героя соперника B
Модель с героем B обыграла героя соперник

Потрясающе, какие мы молодцы.
Теперь представим, что героев у нас не 3, а там, 200. А ещё мы живём в реальном мире, где и везде и повсюду вероятности, так что есть шанс у героя в неравной схватке с другим изредка выйти победителем.

Все эти вероятности мы учтём в нашей матрице, а после этого проверим нашу модель на работоспособность в самых что ни на есть боевых условиях

In [ ]:
hero_number = 200
matrix = [[rnd.random() * 0.8 for _ in range(hero_number)] for __ in range(hero_number)]
for i in range(hero_number):
    matrix[i][i] = 0.5
    for j in range(i+1, hero_number):
        matrix[i][j] = 1 - matrix[j][i]

'''for i in range(hero_number):
    for j in range(hero_number):
        print(round(matrix[i][j], 2), end = ' ')
    print()'''

def reward(a, b):
    return 1 if rnd.random() < matrix[a][b] else 0

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape=(1,)),
    keras.layers.Embedding(input_dim=hero_number, output_dim=hero_number+1),
    keras.layers.Flatten(),
    keras.layers.Dense(hero_number, activation='softmax')
])

optimizer = keras.optimizers.Adam(learning_rate=0.01)

num_episodes = 300   # эпохи обучения
steps_per_ep = 500    # матчей за одну эпоху

for episode in range(num_episodes):
    with tf.GradientTape() as tape:
        opp_hero = np.random.randint(0, hero_number, size=steps_per_ep)

        state_input = tf.constant(opp_hero, dtype=tf.int32)
        state_input = tf.reshape(state_input, (steps_per_ep, 1))
        probs = model(state_input, training=True)
        log_probs = tf.math.log(probs + 1e-10)

        sampled_actions = tf.random.categorical(log_probs, num_samples=1)
        sampled_actions = tf.squeeze(sampled_actions, axis=1)

        rewards = []
        for i in range(steps_per_ep):
            model_hero = sampled_actions[i].numpy()
            rewards.append(reward(model_hero, opp_hero[i]))

        rewards = tf.constant(rewards, dtype=tf.float32)
        chosen_action_log_p = tf.reduce_sum(
            tf.one_hot(sampled_actions, depth=hero_number) * log_probs,
            axis=1
        )
        loss = -tf.reduce_mean(rewards * chosen_action_log_p)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if (episode + 1) % 20 == 0:
        mean_reward = tf.reduce_mean(rewards)
        print(f"Episode {episode+1}/{num_episodes} | Mean reward: {mean_reward.numpy():.2f} | Loss: {loss.numpy():.3f}")

Episode 20/300 | Mean reward: 0.48 | Loss: 2.546
Episode 40/300 | Mean reward: 0.52 | Loss: 2.725
Episode 60/300 | Mean reward: 0.52 | Loss: 2.675
Episode 80/300 | Mean reward: 0.57 | Loss: 2.874
Episode 100/300 | Mean reward: 0.58 | Loss: 2.570
Episode 120/300 | Mean reward: 0.63 | Loss: 2.040
Episode 140/300 | Mean reward: 0.75 | Loss: 1.234
Episode 160/300 | Mean reward: 0.81 | Loss: 0.766
Episode 180/300 | Mean reward: 0.83 | Loss: 0.387
Episode 200/300 | Mean reward: 0.85 | Loss: 0.204
Episode 220/300 | Mean reward: 0.85 | Loss: 0.080
Episode 240/300 | Mean reward: 0.85 | Loss: 0.047
Episode 260/300 | Mean reward: 0.86 | Loss: 0.025
Episode 280/300 | Mean reward: 0.86 | Loss: 0.025
Episode 300/300 | Mean reward: 0.85 | Loss: 0.038


In [ ]:
N_test = 1000
print(hero_number)
opp_hero_test = np.random.randint(0,3, size=N_test)
test_input = tf.constant(opp_hero_test, dtype=tf.int32)
test_input = tf.reshape(test_input, (N_test, 1))

test_probs = model(test_input, training=False)  # (N_test,3)
test_log_probs = tf.math.log(test_probs + 1e-10)
test_actions = tf.random.categorical(test_log_probs, num_samples=1)
test_actions = tf.squeeze(test_actions, axis=1).numpy()

wins = 0
L = [chr(i + 100) for i in range(hero_number)]
for i in range(N_test):
    model_hero = test_actions[i]
    opp_hero = opp_hero_test[i]
    r = reward(model_hero, opp_hero)
    wins += r
    if i % 50 == 0:
        if r:
            print(f'Модель с героем {L[model_hero]} обыграла героя соперника {L[opp_hero]}')
        else:
            print(f'x - Модель с героем {L[model_hero]} проиграла герою соперника {L[opp_hero]}')

win_percent = 100.0 * wins / N_test
print(f"\nИтоговая проверка: модель выиграла в {win_percent:.2f}% матчей.")

200
x - Модель с героем   проиграла герою соперника f
Модель с героем é обыграла героя соперника d
Модель с героем ĩ обыграла героя соперника e
Модель с героем ĩ обыграла героя соперника e
Модель с героем   обыграла героя соперника f
Модель с героем   обыграла героя соперника f
Модель с героем é обыграла героя соперника d
Модель с героем ĩ обыграла героя соперника e
Модель с героем   обыграла героя соперника f
Модель с героем   обыграла героя соперника f
Модель с героем ĩ обыграла героя соперника e
x - Модель с героем   проиграла герою соперника f
Модель с героем ĩ обыграла героя соперника e
Модель с героем   обыграла героя соперника f
x - Модель с героем é проиграла герою соперника d
x - Модель с героем é проиграла герою соперника d
x - Модель с героем ĩ проиграла герою соперника e
Модель с героем é обыграла героя соперника d
Модель с героем ĩ обыграла героя соперника e
Модель с героем ĩ обыграла героя соперника e

Итоговая проверка: модель выиграла в 76.50% матчей.


Выгрузим данные из файла

In [ ]:
import ast

with open('matrix.txt', 'r') as f:
    lines = f.readlines()

matrix = ast.literal_eval(lines[0].strip())
hero_names = ast.literal_eval(lines[1].strip())
hero_names_rev = ast.literal_eval(lines[2].strip())

print("Matrix:")
print(matrix)

print("\nHero Names:")
print(hero_names)

print("\nHero Names Reverse:")
print(hero_names_rev)


Matrix:
[[0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.